In [ ]:
import joblib
month_sheets = joblib.load('./month_sheets.pkl')
month_sheet = month_sheets['2021/2']

# 分析&作圖

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 計算月表的支出、收入以及淨額

In [ ]:
expenditure = {k:v for k, v in month_sheet.sum_mainitems().items() if v < 0}
income = {k:v for k, v in month_sheet.sum_mainitems().items() if v > 0}

expenditure_amount = sum(expenditure.values())
income_amount = sum(income.values())

print('支出: {}, 收入: {}, 淨額: {}'.format(expenditure_amount, income_amount, expenditure_amount+income_amount))

### 子類別前N名花費

In [ ]:
N = 10
name_money = ((subitem.name, subitem.money) for subitem in month_sheet.get_subitems())
top_N_subitems = sorted(name_money, key=lambda x:x[-1], reverse=False)[:N]
print(top_N_subitems)

## 月表每日紀錄

In [ ]:
from collections import defaultdict, OrderedDict
history = defaultdict(list)

# 創建一個key為subitem date, value為subitem list的dict
for subitem in month_sheet.get_subitems():
    history[subitem.date].append(subitem)
    
history = OrderedDict(sorted(history.items(), key=lambda x: int(x[0].replace('/', ''))))

### 子項目-早餐的歷史紀錄

In [ ]:
# 初始化fig
fig = go.Figure()
# 建立一個專屬於早餐歷史紀錄的字典
bf_history = {}
# 將早餐的金額加總(因為可能有人一天內吃了複數頓早餐，或是skip掉就要填寫0.0)
for date, subitems in history.items():
    bf_history[date] = list(filter(lambda x: x.name=='早餐', subitems))
    try:
        bf_history[date] = sum([-bf_item.money for bf_item in bf_history[date]])
    except:
        bf_history[date] = 0.0
x, y = zip(*bf_history.items())
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines+markers',
                    name='lines'))
fig.update_layout(title_text='{}月表的每日早餐紀錄'.format(month_sheet.date),
                  xaxis_title='日期',
                 yaxis_title='金額')

### 所有子項目的歷史紀錄
以plotyly作圖的好處之一就是可以跟圖做即時互動，例如下方的stacked bar plot假設不想看某個子項目，僅需在右方圖例(legend)按一下該子項目就可以隱藏/顯示囉~

In [ ]:
import numpy as np
# 初始化fig
fig = go.Figure()
# 製作一個矩陣，row為子類別，col為日期
subitem_names = month_sheet.get_subitems_names()
subitem_init = np.zeros((len(subitem_names), len(history.keys())))
# 待會要把日期與名稱對應填入矩陣中，所以先準備好兩個將str轉換成index的字典
date_to_idx = {key:idx for idx, key in enumerate(history.keys())}
name_to_idx = {key:idx for idx, key in enumerate(subitem_names)}
# 開填!
for date, subitems in history.items():
    for subitem in subitems:
        date_idx = date_to_idx[date]
        name_idx = name_to_idx[subitem.name]
        subitem_init[name_idx][date_idx] += subitem.money
# go.Bar吃的資料是每個子類別在每個日期的金額
for subitem_name in subitem_names:
    fig.add_trace(go.Bar(x=list(history.keys()), y=subitem_init[name_to_idx[subitem_name]], name=subitem_name))

fig.update_layout(title_text='{}月表子項目歷史紀錄'.format(month_sheet.date),
                  xaxis_title='日期',
                  yaxis_title='金額',
                  barmode='stack')
fig.show()

### 主類別圓餅圖

In [ ]:
# 初始化subplots
fig = make_subplots(rows=1, cols=2, 
                    specs=[[{"type": "pie"}, {"type": "pie"}]],
                   subplot_titles=("支出","收入"))
# 將支出與收入分開
expenditure = {k:v for k, v in month_sheet.sum_mainitems().items() if v < 0}
income = {k:v for k, v in month_sheet.sum_mainitems().items() if v > 0}
# 支出的圓餅圖
labels, values = zip(*expenditure.items())
values = tuple(-v for v in values)
fig.add_trace(go.Pie(labels=labels, values=values), row=1, col=1)
# 收入的圓餅圖
labels, values = zip(*income.items())
fig.add_trace(go.Pie(labels=labels, values=values), row=1, col=2)

### 子類別圓餅圖

In [ ]:
# 將月表的所有主類別的各項子類別和展開成一個ordereddict
od = OrderedDict()
for mn in month_sheet.get_mainitems():
    od.update(mn.sum_subitems())

# 剩下的就跟上面的主類別圓餅圖一樣做法~
expenditure = {k:v for k, v in od.items() if v < 0}
income = {k:v for k, v in od.items() if v > 0}

fig = make_subplots(rows=1, cols=2, 
                    specs=[[{"type": "pie"}, {"type": "pie"}]],
                   subplot_titles=("支出","收入"))

labels, values = zip(*expenditure.items())
values = tuple(-v for v in values)
fig.add_trace(go.Pie(labels=labels, values=values), row=1, col=1)

labels, values = zip(*income.items())
fig.add_trace(go.Pie(labels=labels, values=values), row=1, col=2)

### 上述的圖表在MOZE裡面應該都有，但轉成結構化物件後就方便做自己客製化的表，例如:
可以畫出類似薪水MoM或YoY成長圖，這裡用下午茶來試試看

In [ ]:
# 將每月下午茶的金額抓出來
months = tuple(month_sheets.keys())
afternoonteas_in_month = []
for ms in month_sheets.values():
    try:
        afternoonteas_in_month.append(-ms['飲食'].sum_subitems()['下午茶'])
    except KeyError: # 可能某個月沒有買下午茶
        afternoonteas_in_month.append(0.0)
afternoonteas_in_month = np.array(afternoonteas_in_month)
# 計算每月之間的成長率
afternoonteas_in_month_ratio = afternoonteas_in_month[1:] / afternoonteas_in_month[:-1]

In [ ]:
# 初始化fig
fig = make_subplots(specs=[[{"secondary_y": True}]])
# 將下午茶絕對值畫成bar plot
fig.add_trace(go.Bar(x=months, 
                     y=afternoonteas_in_month,
                     name="下午茶花費"),
                     secondary_y=False)
# 將下午茶MoM成長率畫成line plot
fig.add_trace(go.Scatter(x=months[1:], 
                         y=afternoonteas_in_month_ratio, 
                         mode='lines+markers',
                         name="下午茶月成長率"),
                         secondary_y=True)

fig.update_layout(title_text='下午茶月成長率',
                  xaxis_title='月份')
fig.update_yaxes(title_text='金額', secondary_y=False)
fig.update_yaxes(title_text='成長率', secondary_y=True)